In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

#  Load & Prepare the Dataset

In [ ]:
# # Load .csv-data to DataFrame
df_classes = pd.read_csv("/kaggle/input/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
df_edges = pd.read_csv("/kaggle/input/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
df_features = pd.read_csv("/kaggle/input/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)

# Rename columns: 1."txId" and 2."time_step"
df_features.rename(columns={0: "txId", 1: "time_step"}, inplace=True)

# Change "unknown" to "3" 
df_classes.loc[df_classes["class"] == "unknown", "class"] = 3
df_classes["class"] = df_classes["class"].astype(int)

# Merge "timestamps" column from df_features with classes
df_nodes = df_classes.merge(df_features[["txId", "time_step"]], on="txId")

# Keep edges where at least one transaction is known (class 1 or 2)
valid_txIds = set(df_nodes["txId"])
df_edges_known = df_edges[(df_edges["txId1"].isin(valid_txIds)) | (df_edges["txId2"].isin(valid_txIds))]

# Reset index
df_nodes.reset_index(drop=True, inplace=True)
df_edges_known.reset_index(drop=True, inplace=True)

# Save cleaned data
df_nodes.to_csv("filtered_nodes.csv", index=False)
df_edges_known.to_csv("filtered_edges.csv", index=False)

print("Filtered Nodes:")
print(df_nodes.head())

print("\nFiltered Edges:")
print(df_edges_known.head())

# Time Step 27

**A. Directed Acyclic Graph (DAG)**

In [ ]:
# 1. Graph: all nodes and edge
def plot_dag_for_time_step(time_step):
    # Filter transactions at time step
    tx_at_time = df_nodes[df_nodes["time_step"] == time_step]
    valid_txIds_at_time = set(tx_at_time["txId"])
    
    # Filter edges that involve transactions from this time step
    edges_at_time = df_edges_known[
        (df_edges_known["txId1"].isin(valid_txIds_at_time)) | 
        (df_edges_known["txId2"].isin(valid_txIds_at_time))
    ]
    
    G = nx.DiGraph() # create graph
    # Add nodes
    for _, row in tx_at_time.iterrows():
        G.add_node(row["txId"], tx_class=row["class"]) # Add class attribute
    # Add edges
    G.add_edges_from(edges_at_time[["txId1", "txId2"]].itertuples(index=False, name=None))
    
    # Define node colors
    color_map = {
        1: "red",    # Illicit transactions
        2: "green",  # Licit transactions
        3: "gray"    # Unknown
    }
    node_colors = [color_map[G.nodes[n]["tx_class"]] for n in G.nodes]
    
    plt.figure(figsize=(14, 10))
    pos = nx.spring_layout(G, k=0.3, seed=42) # node positioning
    nx.draw(G, pos, with_labels=False, node_size=100, node_color=node_colors, edge_color="gray", alpha=0.7)
    
    label_pos = {k: (v[0], v[1] + 0.02) for k, v in pos.items()}  # Add labels, offset slightly
    nx.draw_networkx_labels(G, label_pos, font_size=6, font_color="black")

    plt.title(f"Transaction Flow DAG at Time Step {time_step}", fontsize=14)
    plt.show()
    
    return G
G = plot_dag_for_time_step(27)

Following graphs added after step B. (see below)

In [ ]:
# 2.Graph: illicit transactions and one further connected node
def plot_illicit_network(time_step):
    # Filter illicit transactions
    illicit_tx_at_time = df_nodes[(df_nodes["time_step"] == time_step) & (df_nodes["class"] == 1)]
    illicit_txIds_at_time = set(illicit_tx_at_time["txId"])
    # Filter edges, at least one node illicit
    related_edges = df_edges_known[
        (df_edges_known["txId1"].isin(illicit_txIds_at_time)) | 
        (df_edges_known["txId2"].isin(illicit_txIds_at_time))
    ]
    
    # Collect all unique transactions in these edges (both illicit and connected)
    related_txIds = set(related_edges["txId1"]).union(set(related_edges["txId2"]))
    # Get all nodes involved (illicit + their connections)
    related_nodes = df_nodes[df_nodes["txId"].isin(related_txIds)]
    

    G = nx.DiGraph()
    for _, row in related_nodes.iterrows():
        G.add_node(row["txId"], tx_class=row["class"])
    G.add_edges_from(related_edges[["txId1", "txId2"]].itertuples(index=False, name=None))
    
    color_map = {
        1: "red",
        2: "green",
        3: "gray"
    }
    node_colors = [color_map[G.nodes[n]["tx_class"]] for n in G.nodes]
    
    plt.figure(figsize=(14, 10))
    pos = nx.spring_layout(G, k=0.5, seed=42)  # k=compactness
    nx.draw(
        G, pos, with_labels=False, node_size=200, node_color=node_colors, 
        edge_color="gray", alpha=0.5  # alpha=opacity
    )
    label_pos = {k: (v[0], v[1] + 0.05) for k, v in pos.items()}
    nx.draw_networkx_labels(G, label_pos, font_size=12, font_color="black")
    plt.title(f"Illicit Transaction Network at Time Step {time_step}", fontsize=16)
    plt.show()

    return G
G_illicit_network = plot_illicit_network(27)

In [ ]:
# 3.,4.Graph: Illicit transactions outgoing edges & incoming edges
def plot_illicit_edges(time_step):
    # Only illicit nodes and their outgoing edges
    illicit_nodes = [n for n in G.nodes if G.nodes[n]["tx_class"] == 1]  # Illicit nodes
    illicit_outgoing_edges = [(u, v) for u, v in G.edges if u in illicit_nodes]  # from illicit nodes
    illicit_incoming_edges = [(u, v) for u, v in G.edges if v in illicit_nodes]  # to illicit nodes

    # Subgraphs for outgoing and incoming edges
    G_illicit_outgoing = G.edge_subgraph(illicit_outgoing_edges).copy()
    G_illicit_incoming = G.edge_subgraph(illicit_incoming_edges).copy()
    
    # Assign colors
    color_map = {1: "red", 2: "green", 3: "gray"}
    node_colors_outgoing = [color_map[G.nodes[n]["tx_class"]] for n in G_illicit_outgoing.nodes]
    node_colors_incoming = [color_map[G.nodes[n]["tx_class"]] for n in G_illicit_incoming.nodes]
    
    # 3.Graph: outgoing edges
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G_illicit_outgoing, k=0.8, seed=42)  # Adjust layout
    nx.draw(G_illicit_outgoing, pos, with_labels=True, node_size=300, node_color=node_colors_outgoing, edge_color="gray", alpha=0.7)
    plt.title("Illicit Nodes and Their Outgoing Edges", fontsize=16)
    plt.show()
    
    # 4.Graph: incoming edges
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G_illicit_incoming, k=0.8, seed=42)  # Adjust layout
    nx.draw(G_illicit_incoming, pos, with_labels=True, node_size=300, node_color=node_colors_incoming, edge_color="gray", alpha=0.7)
    plt.title("Illicit Nodes and Their Incoming Edges", fontsize=16)
    plt.show()
plot_illicit_edges(27)

*Indication of licit and unknown transactions which could be involved in illegitimate actions:*
* 3. Graph: Multiple illict nodes transferring to one licit or unknown node.  
* 4. Graph: One licit or unknown node transfering to 2 or more illict nodes.  
Both cases could indicate that the licit or unknown transaction could be hidden illict one, e.g. to be a undetected mule or a part of smurfing, where large amounts are broken into smaller ones..

**B. Analyze the transaction flows**

In [ ]:
# List illicit transactions in timestep 27
count = 0
for n in G.nodes:
    if G.nodes[n]["tx_class"] == 1:
        count += 1
        print(f"{count}. Node: {n}, Attributes: {G.nodes[n]}")

**1. Identify Central Nodes (Hubs)**  
Find which transactions are the most connected (with highest degree centrality)

In [ ]:
# 1a. 10 most active transactions, their degree centrality, ranked descendant
central_nodes = sorted(G.degree, key=lambda x: x[1], reverse=True)[:10]
print("Most Connected Transactions (transactionID, in- & out-degree):", central_nodes)

In [ ]:
# 1b. Find if These Transactions are Illicit, show class
for tx, deg in central_nodes:
    print(f"Transaction {tx}: Class {G.nodes[tx]['tx_class']} with {deg} connections")

*Result: 10 most active transactions are all licit or unknown.  
They can be legitimate. Further research: if connected with illict transaction, could be suspected for money laundering or structuring (smurfing), involved in rapid fund movements or being a central hub in illicit fund transfers.*

In [ ]:
# 1c. Rank illicit transactions upon degree centrality
# Illicit transaction nodes
illicit_nodes = [node for node, data in G.nodes(data=True) if data.get("tx_class") == 1]
illicit_degrees = [(node, G.degree(node)) for node in illicit_nodes] # Degree centrality
ranked_illicit_transactions = sorted(illicit_degrees, key=lambda x: x[1], reverse=True) # Sort by degree, descending

# Display
if ranked_illicit_transactions:
    print("Top Illicit Transactions by Degree Centrality:")
    for rank, (tx, degree) in enumerate(ranked_illicit_transactions[:10], start=1):
        print(f"Rank {rank}: Transaction {tx} - Degree {degree}")
else:
    print("No illicit transactions found in the dataset.")

**2. Trace Paths Between Illicit Transactions**  
find suspicious licit transactions by identifying connections between illicit transactions

In [ ]:
# Find edges where both nodes are illicit transactions
illicit_edges = [(u, v) for u, v in G.edges if G.nodes[u]['tx_class'] == 1 and G.nodes[v]['tx_class'] == 1]

# Print illicit-to-illicit connections if they exist, otherwise print a message indicating none
if illicit_edges:
    print("Illicit-to-Illicit Transaction Connections:", illicit_edges)
else:
    print("No illicit-to-illicit connections")

*Result: There are no direct transfers for one illicit transactions to another. This could identify an affiliation to a group.*

**3. Filter for high degree licit nodes, which are connected to illicit nodes**

In [ ]:
suspicious_nodes = [
    n for n in G.nodes 
    if G.degree[n] > 10 and G.nodes[n]['tx_class'] == 2 and 
    any(G.nodes[neighbor]['tx_class'] == 1 for neighbor in G.neighbors(n))
]
print("Licit Transactions with High Connectivity to Illicit Ones:", suspicious_nodes)

*Result: There are no licit nodes with more than 10 conections, connected to illicit nodes.*

**4. Find illicit nodes, that are connected via one or multiple licit nodes**

In [ ]:
illicit_nodes = [n for n in G.nodes if G.nodes[n]['tx_class'] == 1]  # Get illicit transactions
suspicious_paths = []  # Store paths that pass through licit nodes

# Check for illicit-illicit connections through at least one licit node
for source in illicit_nodes:
    for target in illicit_nodes:
        if source != target:
            try:
                path = nx.shortest_path(G, source=source, target=target)
                # Ensure the path contains at least one licit transaction
                if any(G.nodes[n]["tx_class"] == 2 for n in path[1:-1]):
                    suspicious_paths.append(path)
            except nx.NetworkXNoPath:
                pass  # No path found

# Print the suspicious paths
if suspicious_paths:
    print("Illicit Transactions Connected via Licit Transactions:")
    for p in suspicious_paths:
        print(" → ".join(map(str, p)))
else:
    print("No connection found")

*No path found*  
If a path was found, this could indicate a mule account to "clean" transactions.

# References
https://networkx.org/documentation/stable/index.html  
Terminology:  
* smurfing: https://www.investopedia.com/terms/s/smurf.asp
* mule account: https://thepaymentsassociation.org/article/how-criminals-use-mule-accounts-to-launder-their-money/  
Code was generated with the help of ChatGPT